In [ ]:
#gerekli kütüphanelerin yüklenmesi

import os
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from keras.src.legacy.preprocessing.image import ImageDataGenerator
from keras.src.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from keras.src.utils import image_dataset_from_directory
from PIL import Image



In [2]:
data_dir = "C:\\Users\\hp\\PycharmProjects\\Retina_Optik_Disc\\retina_optic_disc"

In [3]:
train_dir = os.path.join(data_dir  + "\\train")
print("Train Directory --> ", os.listdir(train_dir))

validation_dir = os.path.join(data_dir  + "\\val")
print("Validation Directory --> ", os.listdir(validation_dir))

test_dir = os.path.join(data_dir  + "\\test")

print("Test Directory --> ", os.listdir(test_dir))

Train Directory -->  ['healty', 'unhealty']
Validation Directory -->  ['healty', 'unhealty']
Test Directory -->  ['healty', 'unhealty']


In [13]:
def create_dataframe(data_path):
    filepaths = []
    labels = []

    # Ana klasördeki alt klasörleri (folds) listeleme
    folds = os.listdir(data_path)
    for fold in folds:
        # Alt klasördeki görüntülerin yollarını almak için yol oluşturma
        f_path = os.path.join(data_path, fold)
        imgs = os.listdir(f_path)  # Alt klasördeki görüntüleri listeleme

        # Her görüntü için dosya yolunu ve sınıf etiketini ekleme
        for img in imgs:
            img_path = os.path.join(f_path, img)  # Görselin tam dosya yolunu oluşturma
            filepaths.append(img_path)  # Görselin yolunu listeye ekleme
            labels.append(fold)  # Görselin etiketini listeye ekleme

    # Dosya yolları ve etiketlerin pandas Series'e dönüştürülmesi
    fseries = pd.Series(filepaths, name='Filepaths')
    lseries = pd.Series(labels, name='Labels')

    # İki seriyi birleştirip bir DataFrame olarak döndürme
    return pd.concat([fseries, lseries], axis=1)


train_retina_df = create_dataframe(train_dir)
test_retina_df = create_dataframe(test_dir)
validation_retina_df = create_dataframe(validation_dir)

print(train_retina_df)
print(test_retina_df)
print(validation_retina_df)

                                              Filepaths    Labels
0     C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...    healty
1     C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...    healty
2     C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...    healty
3     C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...    healty
4     C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...    healty
...                                                 ...       ...
5354  C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...  unhealty
5355  C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...  unhealty
5356  C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...  unhealty
5357  C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...  unhealty
5358  C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...  unhealty

[5359 rows x 2 columns]
                                              Filepaths    Labels
0     C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...    healty
1     C:\Users\hp\PycharmProjects\Retina_Optik_Disc

In [ ]:
def prepare_image(folder, target=(224,224), quality=85):
    for file in os.listdir(folder):
        if file.lower().endswith(".jpeg"):
            image_path = os.path.join(folder, file)
            with Image.open(image_path) as image:
                resized_image = image.resize(target)
                # Sıkıştırma seviyesini 85 olarak belirleyip kaydet
                resized_image.save(image_path, quality=quality)
                print(f"{file} - Original size: {image.size}, Resized size: {resized_image.size}")

folder = "retina_optic_disc\\test\\healty"
prepare_image(folder)


NORMAL-1017237-1.jpeg - Original size: (512, 496), Resized size: (224, 224)
NORMAL-101880-1.jpeg - Original size: (512, 496), Resized size: (224, 224)
NORMAL-1025847-1.jpeg - Original size: (768, 496), Resized size: (224, 224)
NORMAL-1038998-1.jpeg - Original size: (512, 496), Resized size: (224, 224)
NORMAL-1042462-1.jpeg - Original size: (512, 496), Resized size: (224, 224)
NORMAL-1058176-1.jpeg - Original size: (512, 496), Resized size: (224, 224)
NORMAL-1063128-1.jpeg - Original size: (512, 496), Resized size: (224, 224)
NORMAL-1073137-1.jpeg - Original size: (512, 496), Resized size: (224, 224)
NORMAL-1081406-1.jpeg - Original size: (512, 496), Resized size: (224, 224)
NORMAL-112928-1.jpeg - Original size: (512, 496), Resized size: (224, 224)
NORMAL-119840-1.jpeg - Original size: (512, 496), Resized size: (224, 224)
NORMAL-12494-1.jpeg - Original size: (1536, 496), Resized size: (224, 224)
NORMAL-12494-10.jpeg - Original size: (512, 496), Resized size: (224, 224)
NORMAL-12494-11.j

In [ ]:
def test_prepare_image(test_folder, target=(224,224), quality=85):
    for file in os.listdir(test_folder):
        if file.lower().endswith(".jpeg"):
            image_path = os.path.join(test_folder, file)
            with Image.open(image_path) as image:
                resized_image = image.resize(target)
                # Sıkıştırma seviyesini 85 olarak belirleyip kaydet
                resized_image.save(image_path, quality=quality)
                print(f"{file} - Original size: {image.size}, Resized size: {resized_image.size}")

test_healty_folder = "retina_optic_disc\\test\\healty"
test_prepare_image(test_healty_folder)
test_unhealty_folder="retina_optic_disc\\test\\unhealty"
test_prepare_image(test_unhealty_folder)

CNV-1016042-1.jpeg - Original size: (512, 496), Resized size: (224, 224)
CNV-1016042-2.jpeg - Original size: (512, 496), Resized size: (224, 224)
CNV-1016042-3.jpeg - Original size: (768, 496), Resized size: (224, 224)
CNV-1016042-4.jpeg - Original size: (512, 496), Resized size: (224, 224)
CNV-103044-1.jpeg - Original size: (512, 496), Resized size: (224, 224)
CNV-103044-10.jpeg - Original size: (512, 496), Resized size: (224, 224)
CNV-103044-11.jpeg - Original size: (512, 496), Resized size: (224, 224)
CNV-103044-12.jpeg - Original size: (512, 496), Resized size: (224, 224)
CNV-103044-2.jpeg - Original size: (512, 496), Resized size: (224, 224)
CNV-103044-3.jpeg - Original size: (512, 496), Resized size: (224, 224)
CNV-103044-4.jpeg - Original size: (512, 496), Resized size: (224, 224)
CNV-103044-5.jpeg - Original size: (512, 496), Resized size: (224, 224)
CNV-103044-6.jpeg - Original size: (512, 496), Resized size: (224, 224)
CNV-103044-7.jpeg - Original size: (512, 496), Resized si

In [23]:
def train_prepare_image(train_folder, target=(224,224), quality=85):
    for file in os.listdir(train_folder):
        if file.lower().endswith(".jpeg"):
            image_path = os.path.join(train_folder, file)
            with Image.open(image_path) as image:
                resized_image = image.resize(target)
                # Sıkıştırma seviyesini 85 olarak belirleyip kaydet
                resized_image.save(image_path, quality=quality)
                print(f"{file} - Original size: {image.size}, Resized size: {resized_image.size}")

train_health_folder = "retina_optic_disc\\train\\healty"
train_prepare_image(train_health_folder)
train_unhealty_folder="retina_optic_disc\\train\\unhealty"
train_prepare_image(train_unhealty_folder)

NORMAL-101880-9.jpeg - Original size: (768, 496), Resized size: (224, 224)
NORMAL-104305-1.jpeg - Original size: (1536, 496), Resized size: (224, 224)
NORMAL-104305-2.jpeg - Original size: (512, 496), Resized size: (224, 224)
NORMAL-108522-1.jpeg - Original size: (1536, 496), Resized size: (224, 224)
NORMAL-112928-1.jpeg - Original size: (1536, 496), Resized size: (224, 224)
NORMAL-112928-2.jpeg - Original size: (1536, 496), Resized size: (224, 224)
NORMAL-112928-3.jpeg - Original size: (1536, 496), Resized size: (224, 224)
NORMAL-112928-4.jpeg - Original size: (512, 496), Resized size: (224, 224)
NORMAL-112928-5.jpeg - Original size: (512, 496), Resized size: (224, 224)
NORMAL-112928-6.jpeg - Original size: (512, 496), Resized size: (224, 224)
NORMAL-112928-7.jpeg - Original size: (512, 496), Resized size: (224, 224)
NORMAL-113243-1.jpeg - Original size: (512, 496), Resized size: (224, 224)
NORMAL-114740-1.jpeg - Original size: (512, 496), Resized size: (224, 224)
NORMAL-114740-10.jpe

In [25]:
def val_prepare_image(validation_folder, target=(224,224), quality=85):
    for file in os.listdir(validation_folder):
        if file.lower().endswith(".jpeg"):
            image_path = os.path.join(validation_folder, file)
            with Image.open(image_path) as image:
                resized_image = image.resize(target)
                # Sıkıştırma seviyesini 85 olarak belirleyip kaydet
                resized_image.save(image_path, quality=quality)
                print(f"{file} - Original size: {image.size}, Resized size: {resized_image.size}")

#validation_health_folder = "retina_optic_disc\\val\\healty"
#val_prepare_image(validation_health_folder)
validation_unhealty_folder="retina_optic_disc\\val\\unhealty"
val_prepare_image(validation_unhealty_folder)

CNV-172472-100.jpeg - Original size: (512, 496), Resized size: (224, 224)
CNV-172472-101.jpeg - Original size: (512, 496), Resized size: (224, 224)
CNV-172472-103.jpeg - Original size: (512, 496), Resized size: (224, 224)
CNV-172472-104.jpeg - Original size: (512, 496), Resized size: (224, 224)
CNV-172472-105.jpeg - Original size: (768, 496), Resized size: (224, 224)
CNV-172472-106.jpeg - Original size: (768, 496), Resized size: (224, 224)
CNV-172472-107.jpeg - Original size: (1536, 496), Resized size: (224, 224)
CNV-172472-108.jpeg - Original size: (1536, 496), Resized size: (224, 224)
CNV-172472-110.jpeg - Original size: (512, 496), Resized size: (224, 224)
CNV-172472-111.jpeg - Original size: (512, 496), Resized size: (224, 224)
CNV-172472-112.jpeg - Original size: (512, 496), Resized size: (224, 224)
CNV-172472-113.jpeg - Original size: (512, 496), Resized size: (224, 224)
CNV-172472-114.jpeg - Original size: (512, 496), Resized size: (224, 224)
CNV-172472-115.jpeg - Original size:

In [26]:
final_train_df=pd.concat([train_retina_df],ignore_index=True)
print(final_train_df)
final_test_df = pd.concat([test_retina_df],ignore_index=True)
print(final_test_df)
final_validation_df=pd.concat([validation_retina_df],ignore_index=True)
print(final_validation_df)

                                              Filepaths    Labels
0     C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...    healty
1     C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...    healty
2     C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...    healty
3     C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...    healty
4     C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...    healty
...                                                 ...       ...
5354  C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...  unhealty
5355  C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...  unhealty
5356  C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...  unhealty
5357  C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...  unhealty
5358  C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...  unhealty

[5359 rows x 2 columns]
                                              Filepaths    Labels
0     C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...    healty
1     C:\Users\hp\PycharmProjects\Retina_Optik_Disc